In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Models from sklaearn for regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')


In [34]:
df = pd.read_csv('./data/stud_data.csv')

In [35]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


# Preparing X and y

In [36]:
X = df.drop('math score', axis = 1)
y = df['math score']

In [37]:
X

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [38]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [39]:
# Create column transformer with 3 type of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes('object').columns

In [40]:
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

In [41]:
preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [42]:
X = preprocessor.fit_transform(X)
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [43]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## Creating Evaluation Function

In [44]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2
    

In [45]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest ": RandomForestRegressor(),
    "XGBoost": XGBRegressor(),
    "Catboost": CatBoostRegressor()    
}
model_list = []
r2_list = [] 


In [46]:
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(
        y_train, y_train_pred
    )
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(
        y_test, y_test_pred
    )

    model_name = list(models.keys())[i]
    print(f"Model: {model_name}\n")
    
    print(f"Model Performance for Train")
    print(f"Train MAE:  {model_train_mae:.4f}")
    print(f"Train MSE:  {model_train_mse:.4f}")
    print(f"Train RMSE: {model_train_rmse:.4f}")
    print(f"Train R2:   {model_train_r2:.4f}")

    print("-"*35)

    print(f"Model Performance for Test")

    print(f"Test MAE:   {model_test_mae:.4f}")
    print(f"Test MSE:   {model_test_mse:.4f}")
    print(f"Test RMSE:  {model_test_rmse:.4f}")
    print(f"Test R2:    {model_test_r2:.4f}")
    print("-" * 60)

    model_list.append(model_name)
    r2_list.append(model_test_r2)
    
    print("="*35)
    print("\n")

Model: Linear Regression

Model Performance for Train
Train MAE:  4.1985
Train MSE:  27.5447
Train RMSE: 5.2483
Train R2:   0.8751
-----------------------------------
Model Performance for Test
Test MAE:   4.4183
Test MSE:   30.8866
Test RMSE:  5.5576
Test R2:    0.8759
------------------------------------------------------------


Model: Lasso

Model Performance for Train
Train MAE:  5.1582
Train MSE:  42.3875
Train RMSE: 6.5106
Train R2:   0.8078
-----------------------------------
Model Performance for Test
Test MAE:   5.3929
Test MSE:   47.2041
Test RMSE:  6.8705
Test R2:    0.8103
------------------------------------------------------------


Model: Ridge

Model Performance for Train
Train MAE:  4.1977
Train MSE:  27.5484
Train RMSE: 5.2487
Train R2:   0.8751
-----------------------------------
Model Performance for Test
Test MAE:   4.4155
Test MSE:   30.8757
Test RMSE:  5.5566
Test R2:    0.8759
------------------------------------------------------------


Model: K-Nearest Neigh

# Results

In [47]:
model_list

['Linear Regression',
 'Lasso',
 'Ridge',
 'K-Nearest Neighbors',
 'Decision Tree',
 'Random Forest ',
 'XGBoost',
 'Catboost']

In [48]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model_Name', 'R2_score']).sort_values(by='R2_score' ,ascending=False)

,Model_Name,R2_score
2,Ridge,0.875907
0,Linear Regression,0.875863
7,Catboost,0.852108
5,Random Forest,0.842587
6,XGBoost,0.811816
1,Lasso,0.810281
3,K-Nearest Neighbors,0.772328
4,Decision Tree,0.721890
